In [10]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
import json,os
import threading

llm = init_chat_model("gemini-2.5-flash",model_provider="google_genai")

In [13]:
msg = """
Generate {count} Python programs that implement the {pattern} Design Pattern.
Output must follow these rules:
Add realword usages of the pattern.
Add also some simulation pattern
The result must be a **JSON Array of exactly {count} strings**.
Each string must contain a **complete, standalone Python program**.
No explanations, no comments, no markdown formatting, no citations, no extra text.
Each program must be different from the others. Make sure patterns are represents different systems like banking, healthcare, education, e-commerce, etc.
"""
msg = PromptTemplate(
    template=msg,
    input_variables=["pattern","count"]
)

In [14]:
chain = msg | llm

In [29]:
def generate_code(pattern: str, count: int):
    print(f"Generating {count} code examples for the {pattern} pattern...")
    return chain.invoke({"pattern": pattern, "count": count})

def save_code(output, pattern: str, base_path: str, starting_index=1):
    json_output = json.loads(output.content)
    for i, code in enumerate(json_output):
        os.makedirs(f"{base_path}/{pattern}", exist_ok=True)
        with open(f"{base_path}/{pattern}/{pattern}_pattern_{starting_index+i}.py", "w") as file:
            file.write(code)
    
    print(f"Saved {pattern} pattern examples")

In [30]:

def software_design_patterns(patterns, base_path, pattern_count=10,starting_index=1):
    for pattern in patterns:
        output = generate_code(pattern, pattern_count)
        save_code(output, pattern, base_path,starting_index)

def software_design_patterns_threaded(patterns,base_path,pattern_count=10,starting_index=1):
    threads = []
    for pattern in patterns:
        t = threading.Thread(target=lambda p=pattern: save_code(generate_code(p,pattern_count),p,base_path,starting_index))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()

In [ ]:
patterns = ["Singleton", "Factory", "Observer", "Decorator", "Strategy"]
patterns = ["Observer"]
base_path = "/home/hasinthaka/Documents/Projects/AI/AI Pattern Mining/Pattern Validator/reposistories/test"
software_design_patterns_threaded(patterns,base_path,pattern_count=2,starting_index=30)

Generating 2 code examples for the Observer pattern...
Saved Observer pattern examples


In [ ]:
output = generate_code("Singleton", 1)

In [6]:
json_output = json.loads(output.content)

In [7]:
json_output

['class ShoppingCartManager:\n    _instance = None\n\n    def __new__(cls):\n        if cls._instance is None:\n            cls._instance = super().__new__(cls)\n            cls._instance.items = {}\n            print("New ShoppingCartManager instance created.")\n        else:\n            print("Existing ShoppingCartManager instance returned.")\n        return cls._instance\n\n    def add_item(self, item_id, quantity):\n        if not isinstance(item_id, str) or not isinstance(quantity, int) or quantity <= 0:\n            print("Invalid item_id or quantity.")\n            return\n        self.items[item_id] = self.items.get(item_id, 0) + quantity\n        print(f"Added {quantity} of \'{item_id}\'. Current cart: {self.items}")\n\n    def remove_item(self, item_id, quantity):\n        if not isinstance(item_id, str) or not isinstance(quantity, int) or quantity <= 0:\n            print("Invalid item_id or quantity.")\n            return\n        if item_id in self.items:\n            sel